# QA calculate_annual_energy_cost_residential_v2
This notebook QAs version 2 of calculate_annual_energy_cost_residential (calculate_annual_energy_cost_residential_v2).

In [1]:
import sys
import logging
import numpy as np
import pandas as pd

sys.path.append(os.path.join('..',''))
import config
import lcoc.urdb as urdb
import lcoc.afdc as afdc
import lcoc.processing as proc
import os
from IPython.display import display
import xlwings as xw

## Set parameters

In [2]:
# Specify version of URDB and AFDC datasets to use. If a specific date, this must be specified in the config file. The current setting will download today's version of the datasets.
urdb_afdc_version = 'specific_date' # 'current'

## Load Data

### Load & Process Data - V1

In [3]:
## LOAD DATA ##
if urdb_afdc_version == 'current':
    db = urdb.DatabaseRates() # gets current snapshot of URDB and saves it to db
elif urdb_afdc_version == 'specific_date':
    urdb_path = config.URDB_PATH
    print(config.URDB_PATH)
    db = urdb.DatabaseRates(urdb_path)

print("Total:", db.rate_data.shape)
print("Res:", db.res_rate_data.shape)
print("Com:", db.com_rate_data.shape)

print('filter expired rates')
db.filter_stale_rates(industry='residential')
print("Res:", db.res_rate_data.shape)

db.filter_stale_rates(industry='commercial')
print("Com:", db.com_rate_data.shape)

# update commercial rates with manual changes to demand and energy usage limits (included in description but missing from database fields)
urdb_rates_update = pd.read_excel(os.path.join(config.DATA_PATH,'urdb','urdb_rates_update_202205.xlsx'),sheet_name='changed_rates')
urdb_rates_update.set_index(keys='label',inplace=True)
db.com_rate_data.set_index(keys='label',inplace=True)
db.com_rate_data.update(urdb_rates_update,overwrite=True)
db.com_rate_data = db.com_rate_data.reset_index()

# classify rates by is_tier, is_seasonal, is_TOU, is_ev-specific (residential only)
ev_rate_words_filepath = os.path.join(config.HOME_PATH,'filters','urdb_res_ev_specific_rate_words.txt')

db.classify_rate_structures(industry='residential', 
                            ev_rate_words_file=ev_rate_words_filepath)

db.classify_rate_structures(industry='commercial')

# standardize units of reporting for commercial rates
db.com_rate_preprocessing()

filters_path = os.path.join(config.HOME_PATH,'filters')

# filter demand rates (residential only)
db.filter_demand_rates(industry='residential') 

# filter commercial rates missing critical fields to approx the cost of electricity
db.additional_com_rate_filters()

# filter rates containing certain phrases in filters/
db.filter_on_phrases(industry='residential', filters_path=filters_path)
db.filter_on_phrases(industry='commercial', filters_path=filters_path)

# combine base rate + adjusted rate
db.combine_rates(industry='residential')
db.combine_rates(industry='commercial')

# filter null rates
db.filter_null_rates(industry='residential')
db.filter_null_rates(industry='commercial')

print('size of rate dataframes:')
print("Res:", db.res_rate_data.shape)
print("Com:", db.com_rate_data.shape)

print('res rate structure breakdown')
db.generate_classification_tree_values(industry='residential')

print('com rate structure breakdown')
db.generate_classification_tree_values(industry='commercial')

C:\Users\Jesse Vega-Perkins\Documents\thesis_ev\02_analysis\lcoc-ldevs\data\urdb\usurdb_20220526.csv
Total: (50311, 670)
Res: (11460, 670)
Com: (32503, 670)
filter expired rates
Res: (6151, 670)
Com: (17961, 670)
size of rate dataframes:
Res: (4772, 732)
Com: (7541, 731)
res rate structure breakdown
com rate structure breakdown


{'demand': 4196,
 'no_demand': 3345,
 'demand/tier': 1282,
 'demand/fixed': 2914,
 'no_demand/tier': 1299,
 'no_demand/fixed': 2046,
 'demand/tier/seasonal': 158,
 'demand/tier/no_seasonal': 1124,
 'demand/fixed/seasonal': 517,
 'demand/fixed/no_seasonal': 2397,
 'no_demand/tier/seasonal': 326,
 'no_demand/tier/no_seasonal': 973,
 'no_demand/fixed/seasonal': 320,
 'no_demand/fixed/no_seasonal': 1726,
 'demand/tier/seasonal/tou': 21,
 'demand/tier/seasonal/no_tou': 137,
 'demand/tier/no_seasonal/tou': 13,
 'demand/tier/no_seasonal/no_tou': 1111,
 'demand/fixed/seasonal/tou': 240,
 'demand/fixed/seasonal/no_tou': 277,
 'demand/fixed/no_seasonal/tou': 243,
 'demand/fixed/no_seasonal/no_tou': 2154,
 'no_demand/tier/seasonal/tou': 7,
 'no_demand/tier/seasonal/no_tou': 319,
 'no_demand/tier/no_seasonal/tou': 4,
 'no_demand/tier/no_seasonal/no_tou': 969,
 'no_demand/fixed/seasonal/tou': 96,
 'no_demand/fixed/seasonal/no_tou': 224,
 'no_demand/fixed/no_seasonal/tou': 171,
 'no_demand/fixed/no_

In [4]:
db.calculate_annual_energy_cost_residential(outpath = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-res-rates'))

Complete, 4766 rates included.


### Load & Process Data - V2

In [5]:
## LOAD DATA ##
if urdb_afdc_version == 'current':
    db = urdb.DatabaseRates() # gets current snapshot of URDB and saves it to db
elif urdb_afdc_version == 'specific_date':
    urdb_path = config.URDB_PATH
    print(config.URDB_PATH)
    db = urdb.DatabaseRates(urdb_path)

print("Total:", db.rate_data.shape)
print("Res:", db.res_rate_data.shape)
print("Com:", db.com_rate_data.shape)

print('filter expired rates')
db.filter_stale_rates(industry='residential')
print("Res:", db.res_rate_data.shape)

db.filter_stale_rates(industry='commercial')
print("Com:", db.com_rate_data.shape)

# update commercial rates with manual changes to demand and energy usage limits (included in description but missing from database fields)
urdb_rates_update = pd.read_excel(os.path.join(config.DATA_PATH,'urdb','urdb_rates_update_202205.xlsx'),sheet_name='changed_rates')
urdb_rates_update.set_index(keys='label',inplace=True)
db.com_rate_data.set_index(keys='label',inplace=True)
db.com_rate_data.update(urdb_rates_update,overwrite=True)
db.com_rate_data = db.com_rate_data.reset_index()

# classify rates by is_tier, is_seasonal, is_TOU, is_ev-specific (residential only)
ev_rate_words_filepath = os.path.join(config.HOME_PATH,'filters','urdb_res_ev_specific_rate_words.txt')

db.classify_rate_structures(industry='residential', 
                            ev_rate_words_file=ev_rate_words_filepath)

db.classify_rate_structures(industry='commercial')

# standardize units of reporting for commercial rates
db.com_rate_preprocessing()

filters_path = os.path.join(config.HOME_PATH,'filters')

# filter demand rates (residential only)
db.filter_demand_rates(industry='residential') 

# filter commercial rates missing critical fields to approx the cost of electricity
db.additional_com_rate_filters()

# filter rates containing certain phrases in filters/
db.filter_on_phrases(industry='residential', filters_path=filters_path)
db.filter_on_phrases(industry='commercial', filters_path=filters_path)

# combine base rate + adjusted rate
db.combine_rates(industry='residential')
db.combine_rates(industry='commercial')

# filter null rates
db.filter_null_rates(industry='residential')
db.filter_null_rates(industry='commercial')

print('size of rate dataframes:')
print("Res:", db.res_rate_data.shape)
print("Com:", db.com_rate_data.shape)

print('res rate structure breakdown')
db.generate_classification_tree_values(industry='residential')

print('com rate structure breakdown')
db.generate_classification_tree_values(industry='commercial')

C:\Users\Jesse Vega-Perkins\Documents\thesis_ev\02_analysis\lcoc-ldevs\data\urdb\usurdb_20220526.csv
Total: (50311, 670)
Res: (11460, 670)
Com: (32503, 670)
filter expired rates
Res: (6151, 670)
Com: (17961, 670)
size of rate dataframes:
Res: (4772, 732)
Com: (7541, 731)
res rate structure breakdown
com rate structure breakdown


{'demand': 4196,
 'no_demand': 3345,
 'demand/tier': 1282,
 'demand/fixed': 2914,
 'no_demand/tier': 1299,
 'no_demand/fixed': 2046,
 'demand/tier/seasonal': 158,
 'demand/tier/no_seasonal': 1124,
 'demand/fixed/seasonal': 517,
 'demand/fixed/no_seasonal': 2397,
 'no_demand/tier/seasonal': 326,
 'no_demand/tier/no_seasonal': 973,
 'no_demand/fixed/seasonal': 320,
 'no_demand/fixed/no_seasonal': 1726,
 'demand/tier/seasonal/tou': 21,
 'demand/tier/seasonal/no_tou': 137,
 'demand/tier/no_seasonal/tou': 13,
 'demand/tier/no_seasonal/no_tou': 1111,
 'demand/fixed/seasonal/tou': 240,
 'demand/fixed/seasonal/no_tou': 277,
 'demand/fixed/no_seasonal/tou': 243,
 'demand/fixed/no_seasonal/no_tou': 2154,
 'no_demand/tier/seasonal/tou': 7,
 'no_demand/tier/seasonal/no_tou': 319,
 'no_demand/tier/no_seasonal/tou': 4,
 'no_demand/tier/no_seasonal/no_tou': 969,
 'no_demand/fixed/seasonal/tou': 96,
 'no_demand/fixed/seasonal/no_tou': 224,
 'no_demand/fixed/no_seasonal/tou': 171,
 'no_demand/fixed/no_

In [6]:
db.calculate_annual_energy_cost_residential_v2(outpath = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-res-rates'))

Complete, 4766 rates included.


As shown below, only 1 V2 rate is significantly different than its V1 counterpart (label #539fc1b9ec4f024c27d8aa17). This is because the period 1, tier 1 rate is nan, while the adjustment is 0.007274 (see rate page here: https://apps.openei.org/USURDB/rate/view/539fc1b9ec4f024c27d8aa17#3__Energy). In V2, the p1t1 rate becomes 0.007274 while in V1, it remains nan because in comb_rates, only the adjustment column has NA values filled with 0. V2 does not use columns created by comb_rates, and instead fills NA values in both the rate and adj columns with 0 before conducting the calculations. This is consistent with what is done for the DCFC calculations.

In [7]:
# QA
res_df_v1 = pd.read_csv(os.path.join(config.HOME_PATH,'outputs','cost-of-electricity','urdb-res-rates','res_rates_v1.csv'),low_memory=False)
res_df_v2 = pd.read_csv(os.path.join(config.HOME_PATH,'outputs','cost-of-electricity','urdb-res-rates','res_rates.csv'),low_memory=False)

res_df_QA = res_df_v1.merge(res_df_v2[['label','electricity_cost_per_kwh']],how='left',on='label',suffixes=['_v1','_v2'])
res_df_QA['diff'] = res_df_QA['electricity_cost_per_kwh_v1'] - res_df_QA['electricity_cost_per_kwh_v2']
res_df_QA['check'] = res_df_QA['electricity_cost_per_kwh_v1'] == res_df_QA['electricity_cost_per_kwh_v2']
res_df_QA[res_df_QA['diff']>=1E-10]

,label,eiaid,name,is_default,startdate,enddate,latest_update,utility,sector,description,...,energyrate/period18/tier0,energyrate/period19/tier0,energyrate/period20/tier0,energyrate/period21/tier0,energyrate/period22/tier0,energyrate/period23/tier0,electricity_cost_per_kwh_v1,electricity_cost_per_kwh_v2,diff,check
3778,539fc1b9ec4f024c27d8aa17,18813.0,Residential,NaN,NaN,NaN,2015-03-26 21:26:32,"Village of Theresa, New York (Utility Company)",Residential,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.062849,0.050649,0.0122,False


In [8]:
# code from V1 function
import warnings
res_rates_tier = res_df_v1
avg_costs = []
i = 3768
avg_tier_rates = []
avg_tier_month_rates = []
for p in range(24):
    if p==0:
        tier_rates = []
        for t in range(11):
            rate_str = 'energyrate/period{0}/tier{1}'.format(p,t)
            rate = res_rates_tier.iloc[i][rate_str]
            tier_rates.append(rate)
            #display(tier_rates)
        
        with warnings.catch_warnings(): #supress warnings
            warnings.simplefilter("ignore", category=RuntimeWarning)
            avg_tier_rate = np.nanmean(np.array(tier_rates))  
        
        avg_tier_rates.append(avg_tier_rate)

    elif p==1:
        print(f'period: {p}')
        tier_rates = []
        for t in range(8):
            rate_str = 'energyrate/period{0}/tier{1}'.format(p,t)
            rate = res_rates_tier.iloc[i][rate_str]
            tier_rates.append(rate)
            print(f'tier: {t}')
            display(tier_rates)
        
        with warnings.catch_warnings(): #supress warnings
            warnings.simplefilter("ignore", category=RuntimeWarning)
            avg_tier_rate = np.nanmean(np.array(tier_rates))

        avg_tier_rates.append(avg_tier_rate)

    elif p>=2 and p<6:
        tier_rates = []
        for t in range(5):
            rate_str = 'energyrate/period{0}/tier{1}'.format(p,t)
            rate = res_rates_tier.iloc[i][rate_str]
            tier_rates.append(rate)
            #display(tier_rates)
        
        with warnings.catch_warnings(): #supress warnings
            warnings.simplefilter("ignore", category=RuntimeWarning)
            avg_tier_rate = np.nanmean(np.array(tier_rates))

        avg_tier_rates.append(avg_tier_rate)

    else:
        rate_str = 'energyrate/period{0}/tier0'.format(p)
        rate = res_rates_tier.iloc[i][rate_str]
        avg_tier_rates.append(rate)

period: 1
tier: 0


[0.1064]

tier: 1


[0.1064, 0.1264]

tier: 2


...

[0.1064, 0.1264, nan, nan, nan, nan]

tier: 6


[0.1064, 0.1264, nan, nan, nan, nan, nan]

tier: 7


[0.1064, 0.1264, nan, nan, nan, nan, nan, nan]